In [27]:
import pandas as pd
import random
import numpy as np
import random
import tensorflow as tf
import operator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [28]:
data=pd.read_csv("../HW-2-Machine-reasoning/dataset1.csv")
data.head(10)

,x,f(x)
0,-0.66,18.37
1,2.99,5.00
2,2.65,5.12
3,-1.72,27.31
4,4.33,6.76
5,1.94,6.13
6,-1.02,21.17
7,0.05,13.73
8,-1.77,27.78
9,1.06,8.76


In [29]:
shuffle(data)

,x,f(x)
6694,-0.99,20.88
15305,3.23,5.05
16101,0.62,10.67
15421,3.40,5.16
10312,-3.14,42.75
...,...,...
22579,4.46,7.13
5680,3.85,5.72
14145,4.04,6.09
14943,-2.62,36.61


In [47]:
x_train, x_test, y_train, y_test = train_test_split(data['x'],data['f(x)'], test_size=0.60, random_state=0)

In [53]:
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [32]:
F = ['+','-','*','/']

In [33]:
T = ['x']+ list(range(-5,6))

In [34]:
ran = [F,T]

In [35]:
tf = [True,False]

In [36]:
print(T)

['x', -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]


In [37]:
class Node():
    def __init__(self, *args):
        if(len(args) > 1):
            self.val = args[0]
            self.left = args[1]
            self.right = args[2]
        else:
            self.val = args[0]
            self.left = None
            self.right = None
            
    def insert_left(self, child):
        if (self.left == None):
            self.left = child
        else:
            child.left = self.left
            self.left = child
    def insert_right(self, child):
        if(self.right == None):
            self.right = child
        else:
            child.right = self.right
            self.right = child
    def evaluate(self, x):
        if(self.val in T):
            if(self.val == 'x'):
                return int(x)
            else:
                return int(self.val)
        elif(self.val in F):
            if(self.val == '+'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left + right
            elif(self.val == '-'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left - right
            elif(self.val == '*'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left * right
            elif(self.val == '/'):
                if(self.right.evaluate(x) == 0):
                    return 1
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left / right
    def fitness(self, x, y):
        return self.evaluate(x)-y
#         return abs(self.evaluate(x)-y)

        

In [38]:

def print2D(root, space) :

    if (root == None) :
        return
    # Increase distance between levels
    space += 10

    print2D(root.right, space)

    # Print current node after space
    print()
    for i in range(10, space):
        print(end = " ")
    print(root.val)

    print2D(root.left, space)



In [39]:
def calculate_mse(x_data, y_data, tree):
    total = 0
    for i in range(len(x_data)):
        total += (tree.fitness(x_data[i],y_data[i]) ** 2)
    return total/len(x_data)

In [40]:
def generate_tree_shapes(size):
    root = Node(random.choice(random.choice(ran)))
    for i in range(size):
        if(random.choice(tf)):
            root.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_left(Node(random.choice(random.choice(ran))))
        
        if(random.choice(tf)):
            root.insert_left(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_left(Node(random.choice(random.choice(ran))))

    return root

In [41]:

def generate_tree(size, root):
    if(size == 1):
        root.right = Node(random.choice(T))
        root.left = Node(random.choice(T))
    else:
        #right node
        newNode = random.choice(ran)
        if (newNode == F):
            root.right = Node(random.choice(F))
            generate_tree(size-1, root.right)
        else:
            root.right = Node(random.choice(T))
        
        #left node
        newNode = random.choice(ran)
        if (newNode == F):
            root.left = Node(random.choice(F))
            generate_tree(size-1,root.left)
        else:
            root.left = Node(random.choice(T))
root = Node(random.choice(F))

In [42]:
def mutateHelper():
    mutation = Node(random.choice(F))
    (generate_tree(3, mutation))
    return mutation

def mutate(tree):
    path = []
    path = randomSubTree(tree, path)
    mutation = mutateHelper()
    tree = replaceNode(tree, path, mutation)
    return tree


rl = [-1,1]
tf = [True, True, True, False]
def randomSubTree(node, path):
    keepGoing = random.choice(tf)
    if(keepGoing and len(path) < 3):
        nextNode = random.choice([node.right, node.left])
        if(nextNode == None):
            return path
        if(nextNode == node.right):
            path.append(1)
            randomSubTree(node.right, path)
        else:
            path.append(-1)
            randomSubTree(node.left, path)
    return path

# this function returns a subtree given a specific path
def getNode(node, path):
    if(path == []):
        return node
    if(path[0]==1):
        return getNode(node.right, path[1::])
    else:
        return getNode(node.left, path[1::])

#this function changes a node in one tree to another given node
def replaceNode(nodeA, pathA, nodeB):
    if(pathA == []):
        return nodeB
    elif(pathA[0] == 1):
        return Node(nodeA.val, nodeA.left, replaceNode(nodeA.right, pathA[1::], nodeB))
    elif(pathA[0] == -1):
        return Node(nodeA.val, replaceNode(nodeA.left, pathA[1::], nodeB), nodeA.right)

        

def crossover(treeA, treeB): 
    # determine a random node in each for crossover
    pathA = []
    pathB = []
    pathA = randomSubTree(treeA, pathA)
    pathB = randomSubTree(treeB, pathB)
    # swapping the random node in both trees
    temp = getNode(treeA, pathA)
    treeA = replaceNode(treeA, pathA, getNode(treeB, pathB))
    treeB = replaceNode(treeB, pathB, temp)
    return treeA, treeB
    
            
    

In [ ]:
population = []
for i in range(10):
    tree = Node(random.choice(F))
    generate_tree(10, tree)
    population.append(tree)

epochs = 200

for i in range(epochs):
    new_generation = []
    w = []
    j = 0
    for i in population:
        w.append(calculate_mse(x_train, y_train, i))
    total = sum(w)
    print(min(w))
    for i in range(len(w)):
        w[i] = (total-w[i])/total
    while(j < 10):
        operator = random.choices(['r','m','c'], weights = [2,5,6], k = 1)[0]
        if(operator == 'r'):
            child = random.choices(population, weights = w, k = 1)[0]
            new_generation.append(child)
            j+=1
        elif(operator == 'm'):
            child = random.choices(population, weights = w, k = 1)[0]
            child = mutate(child)
            new_generation.append(child)
            j+=1
        elif(operator == 'c'):
            children = random.choices(population, weights = w, k = 2)
            childA = children[0]
            childB = children[1]
            crossover(childA, childB)
            new_generation.append(childA)
            new_generation.append(childB)
            j+=2
    population = []
    for i in new_generation:
        population.append(i)
    

773.62423693424
773.62423693424
825.8965965549586
825.8965965549586
462.0991856400033
462.0991856400033
462.0991856400033
248.56267363999928
462.0991856400033
462.0991856400033
644.2888877434019
655.7978936400035
658.8116224399972
709.8326134235093
709.8326134235093
709.8326134235093
854.1541136400072
854.1541136400072
671.4048662819723
854.1541136400072
854.1541136400072
517.142838640002
359.5546796844446
653.4909218177723
720.6738497511169
720.6738497511169
409.08545363999787
662.9280348844449
720.6738497511169
720.6738497511169
720.6738497511169
720.6738497511169
854.1541136400072
854.1541136400072
854.1541136400072
847.7775267540278
854.1541136400072
854.1541136400072
854.1541136400072
832.0684916399996
752.7031280399999
374.0717216400024
507.4416736399982
507.4416736399982
507.4416736399982
507.4416736399982
507.4416736399982
507.4416736399982
507.4416736399982
507.4416736399982
549.4756640399984
549.4756640399984
549.4756640399984
409.08545363999787
409.08545363999787
549.4756640

25.86